In [70]:
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Subset
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import os
import PIL.Image as Image
from IPython.display import display
import random
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(device))

cuda:0
NVIDIA GeForce RTX 2080 Ti


In [71]:
dataset_dir = "/Data/federated_learning/large_vlm_distillation_ood/Resnet18_classification/s_cars_ood_adding_ood/"

train_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.ImageFolder(root=dataset_dir+"train", transform = train_tfms)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir+"test", transform = test_tfms)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 32, shuffle=False, num_workers = 2)

dataset3 = torchvision.datasets.ImageFolder(root=dataset_dir+"test_2add", transform = test_tfms)
test_2add_loader = torch.utils.data.DataLoader(dataset3, batch_size = 32, shuffle=False, num_workers = 2)

In [72]:
def add_samples_to_train_dataset(dataset, dataset3, samples_to_add):
    new_indices = random.sample(range(len(dataset3)), samples_to_add)
    new_samples = [dataset3.samples[i] for i in new_indices]  # Note the use of `samples` attribute
    dataset.samples.extend(new_samples)
    dataset.targets.extend([dataset3.targets[i] for i in new_indices])
    return dataset  # Return the updated dataset

In [73]:
def train_model(dataset, dataset3, trainloader, testloader, test_2add_loader, model, criterion, optimizer, scheduler,
                n_epochs=5):
    losses = []
    train_accuracies = []
    test_accuracies = []
    model = model.to(device)
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        model.train()
        for i, data in tqdm(enumerate(trainloader, 0)):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()

        epoch_duration = time.time() - since
        epoch_loss = running_loss / len(trainloader)
        epoch_acc = 100 * running_correct / len(trainloader.dataset)
        print(f"Epoch {epoch + 1}, duration: {epoch_duration:.2f} s, loss: {epoch_loss:.4f}, Train acc: {epoch_acc:.2f}")

        losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        
        scheduler.step(test_acc)
        since = time.time()
        
        # Add samples to train_dataset after each epoch
        dataset = add_samples_to_train_dataset(dataset, dataset3, 5)  # Update the dataset
        trainloader = DataLoader(dataset, batch_size=32, shuffle=True)  # Reinitialize the DataLoader with the updated dataset

    print('Finished Training')
    return model, losses, train_accuracies, test_accuracies

In [74]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_acc = 100.0 * correct / total
    print(f'Test acc: {test_acc:.2f}')
    return test_acc

In [75]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 196)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.01,momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

/home/ariel/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ariel/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model_ft, training_losses, training_accs, test_accs = train_model(dataset, dataset3, trainloader, testloader, test_2add_loader, model_ft, criterion, optimizer, lrscheduler, n_epochs=90)

481it [01:13,  6.58it/s]

Epoch 1, duration: 73.23 s, loss: 3.1771, Train acc: 31.46


Test acc: 1.21


481it [02:51,  2.80it/s]

Epoch 2, duration: 171.70 s, loss: 0.9935, Train acc: 74.95


...

Test acc: 14.49


490it [03:17,  2.49it/s]

Epoch 63, duration: 197.11 s, loss: 0.1068, Train acc: 98.14


Test acc: 15.49


135it [00:53,  2.52it/s]

In [ ]:
df_train_acc, df_test_acc = pd.DataFrame(training_accs, columns='Train_accuracies'), pd.DataFrame(training_accs, columns='Test_accuracies')

In [ ]:
# f, axarr = plt.subplots(2,2, figsize = (12, 8))
# axarr[0, 0].plot(training_losses)
# axarr[0, 0].set_title("Training loss")
# axarr[0, 1].plot(training_accs)
# axarr[0, 1].set_title("Training acc")
# axarr[1, 0].plot(test_accs)

# axarr[1, 0].set_title("Test acc")

## Setting for new experiment

In [ ]:
""" goal: see how many test samples needed to get accuracy convergance in test set
1. Create test to copy set by taking ~half of test set, half of each class and copy to test to copy
2. create empty folders in test names at train, hopefully dataloader wouold not recognize it as error
3.  Create empty lists for Train acc, vall acc and create index of copied data and setting it to zero
4. After each epoch, copy randomly 5 samples from test to copy to relevant folders on train and copy the new 5 instances to anno_train retrain and recalculate 
    train and val accuracies
5. save logs of train val and indices of adding data